In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Autogluon model
This notebook is to carry out training for the Amex Default Prediction competition using AutoGluon as a starting AutoML modeling technique.

We are creating a notebook specifically for training to avoid loading the large test dataset. Instead of using the CSVs provided by the competition, we use the same datasets but in parquet format shared by a user within the competition.

In [3]:
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from IPython.display import Image, display_svg, SVG
#from dtreeviz.trees import *
from autogluon.tabular import TabularDataset, TabularPredictor
import gc

pd.options.display.max_rows = 20
pd.options.display.max_columns = 8

In [4]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000): 
        with pd.option_context("display.max_columns", 1000): 
            display(df)

In [5]:
PATH="amex-data-integer-dtypes-parquet-format/"

In [6]:
# Read the training dataset and the labels
raw_data_df = pd.read_parquet(f'{PATH}/train.parquet')
train_labels_df = pd.read_csv(f'{PATH}/train_labels.csv.zip')

In [7]:
# Taking a look at the inputs
raw_data_df.sample(6)

,customer_ID,S_2,P_2,D_39,...,D_142,D_143,D_144,D_145
4020185,b9cdf83627a38ebbfd8b8eb2c943f631fa18ea567319513808e1be5adfc5e9f6,2018-02-14,0.806505,0,...,NaN,0,0.007989,0
239033,0b1ecd1e9fd72ebf6e3188c581e90de0210c1d3133b231adfe38a3e4d763e9b7,2017-07-08,1.000348,0,...,NaN,0,0.007258,0
538306,191091b92233586dde76a982176a9f9e25a2c39640de2d820aa04a62df21ac14,2017-12-20,0.861135,0,...,NaN,0,0.003216,0
2865396,84405a24538c7f1c023c05e613e5a1332880034c594813c475afdcbe5a807425,2017-11-11,0.603687,1,...,0.14,1,0.009432,1
5375221,f8c0112a130a70118e9b4c3442516f2853bcd01b4fd7156dee22484acabff263,2017-12-21,0.993622,0,...,NaN,0,0.003863,0
1110185,3348d5b84899bcd7b7855e71c5fa1eaca8e8c864824800010c24b32caa34a9d8,2017-04-06,0.570044,0,...,NaN,0,0.008626,0


In [8]:
train_labels_df.head(4)

,customer_ID,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc,0


In [9]:
gc.collect()
raw_data_df.shape, train_labels_df.shape

((5531451, 190), (458913, 2))

In [10]:
# Fill in the missing values. This is not actually needed since AutoGluon fills missing values before training as well
raw_data_df = raw_data_df.bfill(axis='rows').ffill(axis='rows')
raw_data_df.reset_index(inplace=True, drop=True)

The input dataset contains multiple statements per customer, so to get a representative set of values we could either groupby and select get an aggregate across all rows (mean, median, percentiles) or simply select the latest statement (tail). The latest statement gives a good indication of the current state of the customer, so we use that here. 

A follow up would be to check if model performance gets better by computing an aggregate of the row values per customer instead of the latest one below.

In [12]:
train_df = raw_data_df.groupby(['customer_ID'],as_index=False).tail(1)
train_df.reset_index(drop=True, inplace=True)

In [13]:
train_df.shape, train_labels_df.shape, raw_data_df.shape

((458913, 189), (458913, 2), (5531451, 190))

In [14]:
train_df.columns, train_labels_df.columns

(Index(['customer_ID', 'P_2', 'D_39', 'B_1', 'B_2', 'R_1', 'S_3', 'D_41', 'B_3',
        'D_42',
        ...
        'D_136', 'D_137', 'D_138', 'D_139', 'D_140', 'D_141', 'D_142', 'D_143',
        'D_144', 'D_145'],
       dtype='object', length=189),
 Index(['customer_ID', 'target'], dtype='object'))

## Data Preparation
We have been specified some columns as categorical variables in the problem statement. We need to ensure these are specified as strings even though their values are likely integers.

In [16]:
# Convert specific columns to categorical variables
obj_col = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
for col in obj_col:
    train_df[col]=train_df[col].astype('int').astype('str')
    print(train_df[col].unique())

['0' '1' '-1' '2']
['2' '1' '3' '6' '4' '5' '7' '0' '-1']
['1' '0' '-1']
['0' '-1' '1']
['5' '0' '2' '7' '3' '1' '4' '6' '-1']
['0' '-1' '1']
['2' '0' '1']
['0' '3' '4' '1' '2' '5']
['0' '2' '1' '3' '-1']
['-1' '1' '0']
['6' '3' '1' '5' '0' '2' '4' '-1']


In [17]:
train_df = train_df.merge(train_labels_df, how='inner', on="customer_ID")

In [18]:
# subsample subset of data for faster demo, try setting this to much larger values
# Not using these in the fit method below. Using the whole training dataset
#subsample_size = 544041  
#train_data_sample = train_df.sample(n=subsample_size, random_state=0)

In [20]:
hyperparams = {  # hyperparameters of each model type
                   'GBM': {},
                   'XGB': {},
                   'FASTAI': {},
                   'CAT': {},
                   'NN_TORCH': {}, 
                  }

In [21]:
predictor = TabularPredictor(label='target', eval_metric='recall',).fit(train_df,ag_args_fit={'num_gpus': 1}
                                                                 ,hyperparameters=hyperparams
                                                                 , time_limit=4*60*60,presets=["best_quality"])

No path specified. Models will be saved in: "AutogluonModels/ag-20220924_172812/"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 14400s
AutoGluon will save models to "AutogluonModels/ag-20220924_172812/"
AutoGluon Version:  0.5.2
Python Version:     3.8.12
Operating System:   Linux
Train Data Rows:    458913
Train Data Columns: 189
Label Column: target
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Avai

[1000]	valid_set's binary_logloss: 0.234855	valid_set's recall: 0.795207


	Training S1F2 with GPU, note that this may negatively impact model quality compared to CPU training.
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
	Training S1F3 with GPU, note that this may negatively impact model quality compared to CPU training.
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
	Training S1F4 with GPU, note that this may negatively impact model quality compared to CPU training.
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
	Training S1F5 with GPU, note that this may negatively impact model quality compared to CPU training.
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
	Training S1F6 with GPU, note that this may negatively impact model quality compared to CPU training.
[LightGBM] [Fatal] GPU Tree Learne

[1000]	valid_set's binary_logloss: 0.235203	valid_set's recall: 0.797967


	Training S1F7 with GPU, note that this may negatively impact model quality compared to CPU training.
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
	Training S1F8 with GPU, note that this may negatively impact model quality compared to CPU training.
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
	0.7968	 = Validation score   (recall)
	375.29s	 = Training   runtime
	2.81s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ... Training model for up to 14002.72s of the 14002.72s of remaining time.
	Memory not enough to fit CatBoostModel folds in parallel. Will do sequential fitting instead. 	Consider decreasing folds trained in parallel by passing num_folds_parallel to ag_args_ensemble when calling predictor.fit
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	Training S1F1 with GPU, note that this may negatively i

In [5]:
predictor = TabularPredictor.load("AutogluonModels/ag-20220924_172812/")
predictor.leaderboard()

                    model  score_val  pred_time_val     fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  NeuralNetFastAI_BAG_L1   0.844675      20.389701  6280.960492               20.389701        6280.960492            1       True          3
1     WeightedEnsemble_L2   0.844675      21.162452  6411.078235                0.772751         130.117743            2       True          6
2   NeuralNetTorch_BAG_L1   0.831412      75.923898  2341.561825               75.923898        2341.561825            1       True          5
3         LightGBM_BAG_L1   0.796319       6.998059   952.755348                6.998059         952.755348            1       True          1
4          XGBoost_BAG_L1   0.795318      19.879759   884.434290               19.879759         884.434290            1       True          4
5         CatBoost_BAG_L1   0.789376       3.534292   869.855051                3.534292         869.855051            1       True          2

,model,score_val,pred_time_val,fit_time,...,fit_time_marginal,stack_level,can_infer,fit_order
0,NeuralNetFastAI_BAG_L1,0.844675,20.389701,6280.960492,...,6280.960492,1,True,3
1,WeightedEnsemble_L2,0.844675,21.162452,6411.078235,...,130.117743,2,True,6
2,NeuralNetTorch_BAG_L1,0.831412,75.923898,2341.561825,...,2341.561825,1,True,5
3,LightGBM_BAG_L1,0.796319,6.998059,952.755348,...,952.755348,1,True,1
4,XGBoost_BAG_L1,0.795318,19.879759,884.434290,...,884.434290,1,True,4
5,CatBoost_BAG_L1,0.789376,3.534292,869.855051,...,869.855051,1,True,2


## Make predictions

In [6]:
best_predictor = TabularPredictor.load("AutogluonModels/ag-20220924_172812/")
predictor = TabularPredictor.load("AutogluonModels/ag-20220924_024411/")

In [7]:
predictor.get_model_best()

'WeightedEnsemble_L2_FULL'

In [8]:
display_all(predictor.leaderboard(extra_info=True, silent=True))

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order,num_features,num_models,num_models_w_ancestors,memory_size,memory_size_w_ancestors,memory_size_min,memory_size_min_w_ancestors,num_ancestors,num_descendants,model_type,child_model_type,hyperparameters,hyperparameters_fit,ag_args_fit,features,child_hyperparameters,child_hyperparameters_fit,child_ag_args_fit,ancestors,descendants
0,WeightedEnsemble_L2,0.838043,138.328958,5516.244585,0.769108,101.599411,2,False,5,2,1,97,3472,48808468,3472,1011985,2,0,WeightedEnsembleModel,GreedyWeightedEnsembleModel,"{'use_orig_features': False, 'max_base_models': 25, 'max_base_models_per_type': 5, 'save_bag_folds': True}",{},"{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': None, 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'drop_unique': False}","[NeuralNetFastAI_BAG_L1, NeuralNetTorch_BAG_L1]",{'ensemble_size': 100},{'ensemble_size': 77},"{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': None, 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'drop_unique': False}","[NeuralNetFastAI_BAG_L1, NeuralNetTorch_BAG_L1]",[]
1,NeuralNetTorch_BAG_L1,0.836495,94.468498,3396.305953,94.468498,3396.305953,1,False,4,173,48,48,48224884,48224884,1011985,1011985,0,1,StackerEnsembleModel,TabularNeuralNetTorchModel,"{'use_orig_features': True, 'max_base_models': 25, 'max_base_models_per_type': 5, 'save_bag_folds': False}",{},"{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': None, 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'drop_unique': False}","[D_116, B_14, R_4, D_39, R_23, D_119, B_16, S_2.day, D_143, D_47, B_10, D_74, R_28, R_1, D_44, S_15, D_118, R_22, D_128, S_23, S_3, P_4, S_19, D_89, D_117, D_140, D_63, S_6, D_54, R_16, D_72, B_20, D_87, B_19, B_41, D_122, B_13, R_6, S_16, D_102, D_125, B_1, D_83, B_26, D_48, D_130, B_40, D_138, D_51, B_37, D_91, R_12, D_124, B_3, R_9, D_136, S_12, D_114, B_28, R_15, S_7, D_137, S_26, R_20, B_18, P_2, D_43, D_75, D_123, S_18, P_3, B_5, D_144, D_66, D_80, S_24, R_18, D_126, D_60, B_23, R_17, S_2.dayofweek, D_111, D_82, R_2, B_31, S_25, D_129, D_79, R_11, S_13, D_127, D_64, R_10, D_131, B_33...","{'num_epochs': 500, 'epochs_wo_improve': 20, 'activation': 'relu', 'embedding_size_factor': 1.0, 'embed_exponent': 0.56, 'max_embedding_dim': 100, 'y_range': None, 'y_range_extend': 0.05, 'dropout_prob': 0.1, 'optimizer': 'adam', 'learning_rate': 0.0003, 'weight_decay': 1e-06, 'proc.embed_min_categories': 4, 'proc.impute_strategy': 'median', 'proc.max_category_levels': 100, 'proc.skew_threshold': 0.99, 'use_ngram_features': False, 'num_layers': 4, 'hidden_size': 128, 'max_batch_size': 512, 'use_batchnorm': False, 'loss_function': 'auto'}","{'batch_size': 256, 'num_epochs': 9}","{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': ['bool', 'int', 'float', 'category'], 'valid_special_types': None, 'ignored_type_group_special': ['text_ngram', 'text_as_category'], 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'num_gpus': 1}",[],[WeightedEnsemble_L2]
2,NeuralNetFastAI_BAG_L1,0.836007,43.091352,2018.339220,43.091352,2018.339220,1,False,2,173,48,48,580112,580112,17381,17381,0,1,StackerEnsembleModel,NNFastAiTabularModel,"{'use_or

In [9]:
predictor.leaderboard(extra_info=True, silent=True)

,model,score_val,pred_time_val,fit_time,...,child_hyperparameters_fit,child_ag_args_fit,ancestors,descendants
0,WeightedEnsemble_L2,0.838043,138.328958,5516.244585,...,{'ensemble_size': 77},"{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': None, 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'drop_unique': False}","[NeuralNetFastAI_BAG_L1, NeuralNetTorch_BAG_L1]",[]
1,NeuralNetTorch_BAG_L1,0.836495,94.468498,3396.305953,...,"{'batch_size': 256, 'num_epochs': 9}","{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': ['bool', 'int', 'float', 'category'], 'valid_special_types': None, 'ignored_type_group_special': ['text_ngram', 'text_as_category'], 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'num_gpus': 1}",[],[WeightedEnsemble_L2]
2,NeuralNetFastAI_BAG_L1,0.836007,43.091352,2018.339220,...,"{'epochs': 30, 'best_epoch': 7}","{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': ['bool', 'int', 'float', 'category'], 'valid_special_types': None, 'ignored_type_group_special': ['text_ngram', 'text_as_category'], 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'num_gpus': 1}",[],[WeightedEnsemble_L2]
3,LightGBM_BAG_L1,0.806039,26.050655,531.317570,...,{'num_boost_round': 219},"{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': ['bool', 'int', 'float', 'category'], 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'num_gpus': 1}",[],[]
4,XGBoost_BAG_L1,0.803262,42.486201,370.850180,...,{'n_estimators': 144},"{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': ['bool', 'int', 'float', 'category'], 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'num_gpus': 1}",[],[]
5,XGBoost_BAG_L1_FULL,NaN,NaN,12.099546,...,{'n_estimators': 144},"{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': ['bool', 'int', 'float', 'category'], 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'num_gpus': 1}",[],[]
6,WeightedEnsemble_L2_FULL,NaN,NaN,273.585621,...,{'ensemble_size': 77},"{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': None, 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'drop_unique': False}","[NeuralNetTorch_BAG_L1_FULL, NeuralNetFastAI_BAG_L1_FULL]",[]
7,NeuralNetTorch_BAG_L1_FULL,NaN,NaN,100.152001,...,"{'batch_size': 256, 'num_epochs': 0}","{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': ['bool', 'int', 'float', 'category'], 'valid_special_types': None, 'ignored_type_group_special': ['text_ngram', 'text_as_category'], 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'num_gpus': 1}",[],[WeightedEnsemble_L2_FULL]
8,NeuralNetFastAI

## Notes
This notebook carries out the training process and we intentionally separate the testing and generation of predictions in to a separate notebook. Loading the testing file and then generating the predictions take a fair amount of time and memory, so keeping them separate helps to make things run better. 

Inferences using AutoGluon also allows for a separate training and testing process since the models are all written to disk and can be simply loaded later/elsewhere. This makes it very flexible to be run on different platforms.